In [37]:
import os
import gc
import sys
import time
import json 
import random
import logging 
import warnings
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', 500)
import scipy.io as sio
import geopandas as gpd
from ctypes import c_double
# geometry
import shapely.wkt
import shapely.ops
from shapely.wkt import loads
from shapely.ops import substring
from shapely.geometry import Point
import scipy.spatial.distance
# raster
import rasterio as rio
import rasterio.plot 
from rasterio.warp import transform
from osgeo import gdal
from PIL import Image
# plot
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

### dir
home_dir = '/home/bingyu/Documents/spatial_queue' # os.environ['HOME']+'/spatial_queue'
work_dir = '/home/bingyu/Documents/spatial_queue' # os.environ['WORK']+'/spatial_queue'
scratch_dir = '/home/bingyu/Documents/spatial_queue/projects/game/simulation_outputs' # os.environ['OUTPUT_FOLDER']

### user
sys.path.insert(0, home_dir)
import util.haversine as haversine
from model.queue_class import Network, Node, Link, Agent
from game.extract_vehicle_locations import extract_vehicle_locations

import warnings
warnings.filterwarnings('error', message='Creating an ndarray from ragged*')

random_seed = 0
random.seed(random_seed)
np.random.seed(random_seed)

In [38]:
def preparation(random_seed=0, fire_id=None, comm_id=None, vphh=None, visitor_cnts=None, contra_id=None, link_closed_time=None, closed_mode=None, shelter_scen_id=None, scen_nm=None, trace_agent_list=None):
    ### logging and global variables

    project_location = '/projects/bolinas'
    network_file_edges = project_location + '/network_inputs/bolinas_edges_sim.csv'
    network_file_nodes = project_location + '/network_inputs/bolinas_nodes_sim.csv'
    network_file_special_nodes = project_location + '/network_inputs/bolinas_special_nodes.json'
    demand_files = [project_location + '/demand_inputs/od_csv/resident_visitor_od_rs{}_commscen{}_vphh{}_visitor{}.csv'.format(random_seed, comm_id, vphh, visitor_cnts)]
    simulation_outputs = '' ### scratch_folder
   
    if contra_id=='0': cf_files = []
    elif contra_id=='1': cf_files = [project_location + '/network_inputs/bolinas_contraflow.csv']
    else: cf_files = []

    scen_nm = scen_nm
    logging.basicConfig(filename=scratch_dir+simulation_outputs+'/log/{}.log'.format(scen_nm), filemode='w', format='%(asctime)s - %(message)s', level=logging.INFO, force=True)
    logging.info(scen_nm)
    print('log file created for {}'.format(scen_nm))

    ### network
    with open(work_dir + network_file_special_nodes) as special_nodes_file:
        special_nodes = json.load(special_nodes_file)
    network = Network()
    network.dataframe_to_network(project_location=project_location, network_file_edges = network_file_edges, network_file_nodes = network_file_nodes, cf_files = cf_files, special_nodes=special_nodes, scen_nm=scen_nm)
    network.add_connectivity()

    ### demand
    network.add_demand(demand_files = demand_files)
    logging.info('total numbers of agents taken {}'.format(len(network.agents.keys())))

    return {'network': network}, {'scen_nm': scen_nm, 'simulation_outputs': simulation_outputs, 'fire_id': fire_id, 'comm_id': comm_id, 'special_nodes': special_nodes, 'link_closed_time': link_closed_time, 'closed_mode': closed_mode, 'shelter_scen_id': shelter_scen_id, 'trace_agent_list': trace_agent_list}, {'in_fire_dict': {}, 'shelter_capacity_122': 200, 'shelter_capacity_202': 100}

def link_model(t, network, link_closed_time=None, closed_mode=None):
    
    # run link model
    for link_id, link in network.links.items(): 
        link.run_link_model(t, agent_id_dict=network.agents)

    return network

def node_model(t, network, special_nodes=None):
    # only run node model for those with vehicles waiting
    node_ids_to_run = set([link.end_nid for link in network.links.values() if len(link.queue_vehicles)>0])

    # run node model
    for node_id in node_ids_to_run:
        node = network.nodes[node_id] 
        n_t_move, transfer_links= node.run_node_model(t, node_id_dict=network.nodes, link_id_dict=network.links, agent_id_dict=network.agents, node2link_dict=network.node2link_dict, special_nodes=special_nodes)
        
    return network

def one_step(t, data, config, update_data):

    network = data['network']
    
    scen_nm, simulation_outputs, fire_id, comm_id, special_nodes, link_closed_time, closed_mode, shelter_scen_id, trace_agent_list = config['scen_nm'], config['simulation_outputs'], config['fire_id'], config['comm_id'], config['special_nodes'], config['link_closed_time'], config['closed_mode'], config['shelter_scen_id'], config['trace_agent_list']

    in_fire_dict, shelter_capacity_122, shelter_capacity_202 = update_data['in_fire_dict'], update_data['shelter_capacity_122'], update_data['shelter_capacity_202']
    
    ### update link travel time before rerouting
    reroute_freq_dict = {'1': 300, '2': 900, '3': 1800}
    reroute_freq = reroute_freq_dict[comm_id]
    
    ### reset link congested counter
    for link in network.links.values():
        link.congested = 0
        if (t%100==0):
            link.update_travel_time_by_queue_length(network.g)

    if t==0:
        for agent_id, agent in network.agents.items():
            network.agents[agent_id].departure_time = np.random.randint(1, 100)

    ### agent model
    t_agent_0 = time.time()
    stopped_agents_list = []
    for agent_id, agent in network.agents.items():
        ### first remove arrived vehicles
        if agent.status == 'arrive':
            network.agents_stopped[agent_id] = (agent.status, t, agent.agent_type)
            stopped_agents_list.append(agent_id)
            continue
        ### find congested vehicles: spent too long in a link
        current_link = network.links[network.node2link_dict[(agent.current_link_start_nid, agent.current_link_end_nid)]]
        if (current_link.link_type != 'v') and (agent.current_link_enter_time is not None) and (t-agent.current_link_enter_time>3600*0.5):
            current_link.congested += 1
            if (shelter_scen_id=='0'):
                if (t-agent.current_link_enter_time>3600*3):
                    agent.status = 'shelter_a1'
            elif shelter_scen_id=='1':
                pass
            else:
                pass
        ### agents need rerouting
        # initial route 
        if (t==0) or (t%reroute_freq==agent_id%reroute_freq) and (agent_id not in trace_agent_list):
            routing_status = agent.get_path(t, g=network.g)
            agent.find_next_link(node2link_dict=network.node2link_dict)
            if agent_id == 0: print(agent.route)
        agent.load_vehicle(t, node2link_dict=network.node2link_dict, link_id_dict=network.links)
        ### remove passively sheltered vehicles immediately, no need to wait for node model
        if agent.status in ['shelter_p', 'shelter_a1', 'shelter_park']:
            current_link.queue_vehicles = [v for v in current_link.queue_vehicles if v!=agent_id]
            current_link.run_vehicles = [v for v in current_link.run_vehicles if v!=agent_id]
            network.nodes[agent.current_link_end_nid].shelter_counts += 1
            network.agents_stopped[agent_id] = (agent.status, t, agent.agent_type)
            stopped_agents_list.append(agent_id)
    for agent_id in stopped_agents_list:
        del network.agents[agent_id]
    t_agent_1 = time.time()

    ### link model
    ### Each iteration in the link model is not time-consuming. So just keep using one process.
    t_link_0 = time.time()
    network = link_model(t, network, link_closed_time=link_closed_time, closed_mode=closed_mode)
    t_link_1 = time.time()
    
    ### node model
    t_node_0 = time.time()
    network = node_model(t, network, special_nodes=special_nodes)
    t_node_1 = time.time()
        
    # stop
    if len(network.agents)==0:
        logging.info("all agents arrive at destinations")
        return network, 'stop'
    else:
        return network, 'continue'

In [39]:
def plot_run_queue_fire(t, current_link = None, fire_id=None, comm_id=None, shelter_scen_id=None, scen_nm=None, roads_gdf=None, fire_raster=None, fire_raster_extent=None, link_stats_gdf=None):
    fig, ax = plt.subplots(1,1, figsize=(10,10))
    ### road network
    roads = roads_gdf.plot(ax=ax, lw=0.5, color='gray', alpha=0.5)
#     print(type(roads_gdf), roads_gdf.head())
    ### current position
    current_link_plot = current_link.plot(ax=ax, lw=3, color='purple', alpha=1)
#     print(type(current_link), current_link.head())
    ### flames
    if (fire_raster is not None):
        cax = fig.add_axes([0.2, 0.2, 0.6, 0.02])
        cmap = cm.get_cmap('Oranges', 5)
        cmap.set_under(color='white', alpha=100)
        flames = ax.imshow(fire_raster, extent=fire_raster_extent, cmap=cmap, vmin=0, vmax=5)
        fig.colorbar(flames, orientation='horizontal', cax=cax)
    ### queue plot
    queue_gdf = link_stats_gdf[link_stats_gdf['q']>0].copy()
    queue_gdf['geometry'] = queue_gdf.apply(lambda x: shapely.ops.substring( x['geometry'], 1-x['q']*8/(x['length']*x['lanes']), 1, normalized=True), axis=1)
    queue = queue_gdf.plot(ax=ax, lw=1, color='red')
    ### run plot
    run_gdf = link_stats_gdf[link_stats_gdf['r']>0].copy()
    run_gdf['geometry'] = run_gdf.apply(lambda x: shapely.ops.substring( x['geometry'], 0, x['r']*8/(x['length']*x['lanes']), normalized=True), axis=1)
    run = run_gdf.plot(ax=ax, lw=1, color='blue')
    (xlim_1, xlim_2) = ax.get_xlim()
    ax.set_xlim([xlim_1 + (xlim_2-xlim_1)*0.1, xlim_1 + (xlim_2-xlim_1)*0.8])
    (ylim_1, ylim_2) = ax.get_ylim()
    ax.set_ylim([ylim_1 + (ylim_2-ylim_1)*0.1, ylim_1 + (ylim_2-ylim_1)*0.6])
    shelter_text = 'no shelter'# {1: 'No sheltering', 2: 'Sheltering'}[shelter_scen_id]
    ax.text(0.6, 0.75, 'Fire location {}\nComm. scenario {}\n{}\n{:.1f} Hr'.format(fire_id, comm_id, shelter_text, t/3600), fontsize=22, transform=ax.transAxes)
#     plt.show()
#     plt.savefig('../visualization_outputs/python_map/{}_t{}.png'.format(scen_nm, t))
#     plt.close()

In [40]:
fire_id='1' # '1', '2', '3'
comm_id='1' # '1', '2', '3'
vphh=2 # vehicles per household
visitor_cnts=300
contra_id='0'
shelter_scen_id='0'
link_closed_time=0
closed_mode='flame'

# base network as the base layer of plotting
roads_df = pd.read_csv(home_dir + '/projects/bolinas/network_inputs/bolinas_edges_sim.csv')
roads_gdf = gpd.GeoDataFrame(roads_df, crs='epsg:4326', geometry=roads_df['geometry'].map(loads)).to_crs(26910)

# set scenario name
scen_nm = "r{}_fire{}_comm{}_vphh{}_vistor{}_contra{}_close{}m{}_shelter{}".format(random_seed, fire_id, comm_id, vphh, visitor_cnts, contra_id, link_closed_time, closed_mode, shelter_scen_id)

In [43]:
# reset random seed
random_seed = 0
random.seed(random_seed)
np.random.seed(random_seed)

trace_agent_id = 0
trace_agent_list = [trace_agent_id]

data, config, update_data = preparation(random_seed=random_seed, fire_id=fire_id, comm_id=comm_id, vphh=vphh, visitor_cnts=visitor_cnts, contra_id=contra_id, shelter_scen_id=shelter_scen_id, link_closed_time=link_closed_time, closed_mode=closed_mode, scen_nm=scen_nm, trace_agent_list=trace_agent_list)

link_speed_dict = dict()
link_vehicles_dict = dict()
link_maxspeed_array = np.array([link.length/(link.fft+0.00001) for link_id, link in data['network'].links.items() if link.link_type != 'v'])
link_length_array = np.array([(link.length+0.00001) for link_id, link in data['network'].links.items() if link.link_type != 'v'])
link_id_array = np.array([link_id for link_id, link in data['network'].links.items() if link.link_type != 'v'])
print(link_id_array[0:10])

for t in range(0, 1200):
    # run simulation for one step
    network, status = one_step(t, data, config, update_data)
    link_vehicles_array = np.array([(len(link.run_vehicles) + len(link.queue_vehicles)) for link_id, link in data['network'].links.items() if link.link_type != 'v'])
    link_density_array = link_vehicles_array * 8 / link_length_array
    link_speed_array = np.where(link_density_array>=1, 0, link_maxspeed_array*(1-link_density_array))
#     print(link_speed_array[0:10])
    link_speed_dict[t] = link_speed_array
    link_vehicles_dict[t] = link_vehicles_array
print(link_speed_dict[199][0:10])

log file created for r0_fire1_comm1_vphh2_vistor300_contra0_close0mflame_shelter0
# agents from file  1490
[0 1 2 3 4 5 6 7 8 9]
{70: 93, 93: 98, 98: 16, 16: 23, 23: 140, 140: 137, 137: 135, 135: 61, 61: 134, 134: 202, 202: 20, 20: 22, 22: 183, 183: 248, 248: 245, 245: 193, 193: 223, 223: 194, 194: 200, 200: 8, 8: 5, 5: 4, 4: 84, 84: 187, 187: 155}
[11.17565734 11.17565734 11.17566356 11.17566356 11.17566176 11.17566176
 11.17567322  8.13337363 11.17557002 11.17557002]


In [51]:
def get_route(g, origin, destin):
    sp = g.dijkstra(origin, destin)
    sp_dist = sp.distance(destin)
    if (sp_dist>1e8):
        sp.clear()
        print('no path found')
        return {}
    else:
        sp_route = sp.route(destin)
        route = {}
        for (start_nid, end_nid) in sp_route:
            route[start_nid] = end_nid
        sp.clear()
        return route

def get_vehicle_position(network, player_origin, player_destin, play_step_duration):
    
    current_link = 'n{}_vl'.format(player_origin)
    current_link_distance = 0
    current_link_angle = 0
    
    for t_p in range(play_step_duration):
        if t_p > 0:
            current_link_distance += link_speed_dict[t_p][current_link] ### the link_speed_dict sequence may not be the same as the link_id sequence
        # print(t_p, current_link_distance)
        if (t_p == 0) or (current_link_distance >= network.links[current_link].length):
            ### list alternative links
            next_links = network.nodes[network.links[current_link].end_nid].outgoing_links
            alternative_links = []
            ### get the angle and numbers of queuing vehicles of alternative links
            for alt_link_id in next_links:
                alternative_link_angle = (current_link_angle - network.links[alt_link_id].in_angle)/3.14*180
                if alternative_link_angle<-180: alternative_link_angle += 360
                if alternative_link_angle>180: alternative_link_angle -= 360
                alternative_link_vehicles = link_vehicles_dict[t_p][alt_link_id]
                alternative_links.append([alt_link_id, alternative_link_angle, alternative_link_vehicles])
            # print(alternative_links)
            ### assign a direction to each alternative link
            alternative_links_by_direction = {'forward': None, 'left': None, 'right': None, 'back': None}
            checked_outgoing_links = []
            # forward
            (forward_link, angle, veh_count) = min(alternative_links, key = lambda l: abs(l[1]-0))
            if abs(angle-0)<=45 and (forward_link not in checked_outgoing_links):
                alternative_links_by_direction['forward'] = [forward_link, veh_count]
                checked_outgoing_links.append(forward_link)
            # left
            (left_link, angle, veh_count) = min(alternative_links, key = lambda l: abs(l[1]-(-90)))
            if abs(angle-(-90))<=45 and (left_link not in checked_outgoing_links):
                alternative_links_by_direction['left'] = [left_link, veh_count]
                checked_outgoing_links.append(left_link)
            # right
            (right_link, angle, veh_count) = min(alternative_links, key = lambda l: abs(l[1]-90))
            # print(right_link, angle)
            if abs(angle-90)<=45 and (right_link not in checked_outgoing_links):
                alternative_links_by_direction['right'] = [right_link, veh_count]
                checked_outgoing_links.append(right_link)
            # backward
            (back_link, angle, veh_count) = max(alternative_links, key = lambda l: abs(l[1]-0))
            if abs(angle-0)>135 and (back_link not in checked_outgoing_links):
                alternative_links_by_direction['back'] = [back_link, veh_count]
                checked_outgoing_links.append(back_link)
            print(alternative_links_by_direction)
            
            ### user incf_filesput of the direction
            agent_next_link_direction = input('Choose from the following actions: ' + ', '.join([k for k, v in alternative_links_by_direction.items() if v is not None]))
            agent_next_link = alternative_links_by_direction[agent_next_link_direction][0]
            
            ### new route based on the chosen direction
            # close all other directions
#             print(alternative_links)
            for [link_id, _, _] in alternative_links:
                if link_id != agent_next_link:
                    print('close {}'.format(link_id))
                    link = network.links[link_id]
                    network.g.update_edge(link.start_nid, link.end_nid, c_double(1e8))

            player_route_by_nodes = get_route(network.g, network.links[current_link].end_nid, player_destin)
            player_route_by_links = [network.node2link_dict[(k,v)] for k,v in player_route_by_nodes.items()]
            # print(player_route)

            for [link_id, _, _] in alternative_links:
                if link_id != agent_next_link:
                    link = network.links[link_id]
                    network.g.update_edge(link.start_nid, link.end_nid, c_double(link.fft))
            data['network'] = network
            
            ### move agent to to chosen link
            current_link = player_route_by_links[0]
            current_link_distance = 0
            current_link_angle = network.links[current_link].out_angle
            print('new link {}\n'.format(current_link))
        
    
player_origin, player_destin = 70, 155
play_step_duration = 500
get_vehicle_position(network, player_origin, player_destin, play_step_duration)

548 95.72041430986432
{'forward': None, 'left': None, 'right': [548, 0], 'back': None}
Choose from the following actions: rightright
new link 548

546 58.702565235053726
{'forward': [515, 0], 'left': None, 'right': [546, 1], 'back': [547, 0]}
Choose from the following actions: forward, right, backright
close 515
close 547
new link 546

90 96.87357795913834
{'forward': [130, 15], 'left': None, 'right': [90, 0], 'back': [545, 0]}
Choose from the following actions: forward, right, backforward
close 90
close 545
new link 130

132 74.21153775987382
{'forward': None, 'left': None, 'right': [132, 8], 'back': None}
Choose from the following actions: rightright
new link 132

572 92.24210570887789
{'forward': None, 'left': [573, 0], 'right': [572, 8], 'back': [131, 0]}
Choose from the following actions: left, right, backright
close 131
close 573
new link 572

570 -1.3486416828307715
{'forward': [570, 18], 'left': [413, 5], 'right': None, 'back': [571, 0]}
Choose from the following actions: forwa